In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2


In [ ]:
load_dotenv()

In [ ]:
key = os.getenv("OPENAI_API_KEY")

In [ ]:
print(key)

In [ ]:
llm = ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo", temperature=0.3)

In [ ]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [ ]:
"""
number=5 
subject="data science"
tone="simple"
"""

In [ ]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [ ]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )


In [ ]:
quiz_generation_prompt

In [ ]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [ ]:
quiz_chain

In [ ]:
TEMPLATE="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [ ]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [ ]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [ ]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [ ]:
"""pdf_reader=PyPDF2.PdfReader(r"C:\Complete_Content\All_Project\TEST_FOR_EVERYTHING\langchain\experiments\Big Data Assessment.pdf")
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()"""

In [ ]:
from pathlib import Path
file_path=r"C:\Complete_Content\All_Project\TEST_FOR_EVERYTHING\langchain\test.txt"

In [ ]:
file_path

In [ ]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [ ]:
TEXT


In [ ]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

In [ ]:
"""mcq_count=5
grade=1
tone="simple"""

In [ ]:
NUMBER=5 
SUBJECT="data science"
TONE="simple"


In [ ]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

In [ ]:
response

In [ ]:
quiz_str=response.get("quiz")

In [ ]:
quiz_dict = json.loads(quiz_str)

In [ ]:
quiz_dict

In [ ]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
df=pd.DataFrame(quiz_table_data)

In [ ]:
df

In [ ]:
df.to_csv("Data_Science_Quiz.csv",index=False)

In [ ]:
import traceback
try:
    c=1/0
except Exception as e:
        traceback.print_exception(type(e), e, e.__traceback__)

In [ ]:
import traceback
try:
    c=1/0
except Exception as e:
    tb=e.__traceback__

In [ ]:
while tb is not None:
    frame = tb.tb_frame
    filename = frame.f_code.co_filename
    lineno = frame.f_lineno
    function_name = frame.f_code.co_name

    print(f"File '{filename}', line {lineno}, in {function_name}")
    tb = tb.tb_next  # Move to the next frame in the traceback